## K26 - Heated Wall - Part 1: Simulation

Interface at 90°.  
Equal fluid densities
Also no Heat capacity => infinitely fast heat conduction  
Height of the domain is reduced  

#### Instructions

This worksheet serves as a basis to conduct various parameter studies for the Heated Wall setup.


### Step 1 - Initialization

Load the BoSSS code, do not change

In [ ]:
//#r "..\..\..\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

### Step 2 - Workflowmanagement, Batchprocessor and Database

In [ ]:
ExecutionQueues

In [ ]:
string ProjectName = $"HeatedWall_Validation";
BoSSSshell.WorkflowMgm.Init(ProjectName);

In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

In [ ]:
myDb.Path

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

### Step 3 - Setup Simulationcontrols

Here we can adapt stuff manually.

In [ ]:
using BoSSS.Application.XNSFE_Solver;

#### Step 3.1 - Define parameter variations

When introducing new variations you need to modify the setup of the controlfiles accordingly.

In [ ]:
Dictionary<string, object[]> Parameters = new Dictionary<string, object[]>();

A few examples of possible variations, the loop creating the control objects contains a few more.  
However not all of them are used here. This can be taken as some sort of blueprint for possible variations.

In [ ]:
Parameters.Clear();
int[] hRes = {16, 32, 64, 128, 256};
Parameters.Add("GridRes", hRes.Select(s => (object)s).ToArray());

int[] pDeg = {2}; // only Temperature order is changed
Parameters.Add("DgDegree", pDeg.Select(s => (object)s).ToArray());

double[] Q = {0.0, 0.002, 0.02};//, 0.2};
Parameters.Add("HeatFlux", Q.Select(s => (object)s).ToArray());

double[] rhoRatio = {1.0};
Parameters.Add("DensityRatio", rhoRatio.Select(s => (object)s).ToArray());

Define a simple routine to try and take the parameter value from the dictionary and if not present some default value.

In [ ]:
public static object GetKey(this Dictionary<string, object[]> Parameters, string key,  Dictionary<string, int> indices, object defaultVal){
    if(Parameters.TryGetValue(key, out var values)){
        return values[indices[key]];
    }else{
        return defaultVal;
    }
}

#### Step 3.2 - Control Creation

Prototype of nested loop over all Parameter lists defined in the Dictionary

In [ ]:
Dictionary<string, int> indices = new Dictionary<string, int>();
indices.Clear();
Parameters.ForEach(kvp => indices.Add(kvp.Key, 0));
indices.Add("stop", -1);

int[] lengths = Parameters.Select(kvp => kvp.Value.Length).ToArray().Cat(-1);
var keys = indices.Keys.ToArray();

int pointer = 0;
while(indices["stop"] == -1){
    
    // for(int i = 0; i < indices.Keys.Count - 1; i++){
    //     Console.Write(keys[i] + ":" +Parameters[keys[i]][indices[keys[i]]]);
    //     Console.Write(";");
    // }
    // Console.WriteLine();

    indices[keys[0]]++;    
    while (indices[keys[pointer]] == lengths[pointer]) {
        indices[keys[pointer]] = 0;
        indices[keys[++pointer]]++;
        if (indices[keys[pointer]] != lengths[pointer])
            pointer = 0;
    }
}

Create the Control Objects in all possible combinations.

In [ ]:
// List of controls
List<XNSFE_Control> Controls = new List<XNSFE_Control>();

// objects to control the nested loop
// ============================================================================
Dictionary<string, int> indices = new Dictionary<string, int>();
indices.Clear();
Parameters.ForEach(kvp => indices.Add(kvp.Key, 0));
indices.Add("stop", -1);

int[] lengths = Parameters.Select(kvp => kvp.Value.Length).ToArray().Cat(-1);
var keys = indices.Keys.ToArray();

int pointer = 0;
// ============================================================================

while(indices["stop"] == -1){
    
    var ctrl = new XNSFE_Control();

    // Add case identifications
    Parameters.ForEach(kvp => ctrl.Paramstudy_CaseIdentification.Add(new Tuple<string, object>(kvp.Key, kvp.Value[indices[kvp.Key]])));


    ctrl.ProjectName = ProjectName;
    ctrl.SessionName = ProjectName;
    Parameters.ForEach(kvp => ctrl.SessionName = ctrl.SessionName  + "_" + kvp.Key + "_" + kvp.Value[indices[kvp.Key]]);

    ctrl.DbPath      = null;
    ctrl.SetDatabase(myDb);
    ctrl.savetodb = true;        

    int p = (int)Parameters.GetKey("DgDegree", indices, 2);
    ctrl.SetDGdegree(p);

    #region grid
    double L = 5.0;
    int kelemR = (int)Parameters.GetKey("GridRes", indices, null);
    string[] Bndy = new string[] {  "Inner",
                                    "NavierSlip_linear_ConstantHeatFlux_right",
                                    "pressure_outlet_ZeroGradient_top",
                                    "freeslip_ZeroGradient_left",
                                    "pressure_outlet_ZeroGradient_bottom"};

    ctrl.GridFunc = delegate () {
        double[] Xnodes = GenericBlas.Linspace(-L, 0, kelemR + 1);
        double[] Ynodes = GenericBlas.Linspace(0, L, kelemR + 1);
        var grd = Grid2D.Cartesian2DGrid(Xnodes, Ynodes);

        for(byte i= 1; i < Bndy.Count(); i++) {
            grd.EdgeTagNames.Add(i, Bndy[i]);
        }

        grd.DefineEdgeTags(delegate (double[] X) {
            byte et = 0;
            if(Math.Abs(X[0] - Xnodes.Last()) < 1e-8)
                return 1;
            if(Math.Abs(X[0] - Xnodes.First()) < 1e-8)
                return 3;
            if(Math.Abs(X[1] - Ynodes.Last()) < 1e-8)
                return 2;
            if(Math.Abs(X[1] - Ynodes.First()) < 1e-8)
                return 4;
            return et;
        });

        return grd;
    };
    #endregion

    #region material
    double densityRatio = (double)Parameters.GetKey("DensityRatio", indices, 1.0);
    ctrl.PhysicalParameters = new BoSSS.Solution.XNSECommon.PhysicalParameters() {
        rho_A = 1.0,
        rho_B = 1.0 * densityRatio,

        mu_A = 1,
        mu_B = 0.001,

        Sigma = 1.0,
        betaS_A = 1000, // sliplength is mu/beta
        betaS_B = 1000,
    };

    ctrl.ThermalParameters = new BoSSS.Solution.XheatCommon.ThermalParameters() {
        rho_A = 1.0,
        rho_B = 1.0 * densityRatio,

        k_A = 1.0,
        k_B = 1.0,

        c_A = 0.0,
        c_B = 0.0,

        hVap = 1,
        T_sat = 0.0
    };

    ctrl.PhysicalParameters.IncludeConvection = true;
    ctrl.ThermalParameters.IncludeConvection = true;
    ctrl.PhysicalParameters.Material = false;
    ctrl.MaterialAtContactLine = (bool)Parameters.GetKey("MaterialAtContactline", indices, false);
    #endregion

    #region Initial Condition - Exact Solution

    // solution for massflux and velocity at level set
    double y0 = 0.2 * L;

    // inital values
    double g = (double)Parameters.GetKey("Gravity", indices, 4.0);
    ctrl.AddInitialValue("Phi", $"(X, t) => -{y0} + X[1]", true);
    ctrl.AddInitialValue("Temperature#A", $"(X, t) => {ctrl.ThermalParameters.T_sat}", true);
    ctrl.AddInitialValue("Temperature#B", $"(X, t) => {ctrl.ThermalParameters.T_sat}", true);
    ctrl.AddInitialValue("GravityY#A", $"(X, t) => -{g}", true);

    #endregion

    #region Boundary Conditions

    double v = 1.0;
    double q = (double)Parameters.GetKey("HeatFlux", indices, 0.0);
    ctrl.AddBoundaryValue(Bndy[1], "HeatFluxX#A", $"(X, t) => {q}", true);
    ctrl.AddBoundaryValue(Bndy[1], "VelocityY#A", $"(X, t) => {v}", true);
    ctrl.AddBoundaryValue(Bndy[1], "VelocityY#B", $"(X, t) => {v}", true);


    ctrl.AddBoundaryValue(Bndy[3]);
    ctrl.AddBoundaryValue(Bndy[2]);
    ctrl.AddBoundaryValue(Bndy[4], "Pressure#A", $"(X, t) => {y0} * {ctrl.PhysicalParameters.rho_A} * {g}", true);

    #endregion

    #region AMR

    // No AMR
    int level = (int)Parameters.GetKey("AMRLevel", indices, 0);
    ctrl.AdaptiveMeshRefinement = level > 0;
    ctrl.activeAMRlevelIndicators.Add(new BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater.AMRonNarrowband() { maxRefinementLevel = level });
    ctrl.AMR_startUpSweeps = level;

    #endregion

    #region Timestepping

    ctrl.AdvancedDiscretizationOptions.SST_isotropicMode = BoSSS.Solution.XNSECommon.SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    ctrl.Option_LevelSetEvolution = (BoSSS.Solution.LevelSetTools.LevelSetEvolution)Parameters.GetKey("LevelSetEvolution", indices, BoSSS.Solution.LevelSetTools.LevelSetEvolution.FastMarching);
    ctrl.Timestepper_LevelSetHandling =  (BoSSS.Solution.XdgTimestepping.LevelSetHandling)Parameters.GetKey("LevelSetHandling", indices, BoSSS.Solution.XdgTimestepping.LevelSetHandling.LieSplitting);

    ctrl.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    ctrl.NonLinearSolver.Globalization = BoSSS.Solution.AdvancedSolvers.Newton.GlobalizationOption.Dogleg;
    ctrl.NonLinearSolver.ConvergenceCriterion = 1e-8;
    ctrl.NonLinearSolver.MaxSolverIterations = 10;

    ctrl.SkipSolveAndEvaluateResidual = false;

    ctrl.TimeSteppingScheme = (BoSSS.Solution.XdgTimestepping.TimeSteppingScheme)Parameters.GetKey("TimesteppingScheme", indices, BoSSS.Solution.XdgTimestepping.TimeSteppingScheme.RK_ImplicitEuler);
    ctrl.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Transient;
    ctrl.dtFixed = 0.01;
    ctrl.Endtime = 0.1;
    ctrl.NoOfTimesteps = int.MaxValue; // timesteps can be adapted, simulate until endtime is reached

    #endregion
    ctrl.PostprocessingModules.Add(new BoSSS.Application.XNSFE_Solver.PhysicalBasedTestcases.MassfluxLogging() { LogPeriod = 1 });
    ctrl.PostprocessingModules.Add(new BoSSS.Application.XNSFE_Solver.PhysicalBasedTestcases.MovingContactLineLogging() { LogPeriod = 1 });

    Controls.Add(ctrl);

    // iteration control
    // ==================================================
    indices[keys[0]]++;    
    while (indices[keys[pointer]] == lengths[pointer]) {
        indices[keys[pointer]] = 0;
        indices[keys[++pointer]]++;
        if (indices[keys[pointer]] != lengths[pointer])
            pointer = 0;
    }
    // ==================================================
}

In [ ]:
Console.WriteLine("Number of Controls: " + Controls.Count + ":");
Controls.ForEach(c => Console.WriteLine("\t" + c.SessionName));

### Step 4 - Start simulations on Batch processor

In [ ]:
var jobs = BoSSSshell.WorkflowMgm.AllJobs;
foreach(var C in Controls) {
    Type solver = typeof(BoSSS.Application.XNSFE_Solver.XNSFE<XNSFE_Control>);

    string jobName       = C.SessionName;
    var oneJob           = new Job(jobName, solver);
    oneJob.NumberOfMPIProcs = 1;    
    oneJob.SetControlObject(C);
    oneJob.Activate(myBatch, false);
}

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(345600, 300); // Block for ~4days adjust as necessary, see ISessionInfo.GetApproximateRunTime() on a finished Session